In [4]:
# import the necessary packages
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2

In [5]:
def findArea(a, b):
     
    # formula to find the
    # area of an Ellipse.
    Area = 3.142 * a * b ;
     
    # Display the result
    return Area

In [9]:
def mouth_aspect_ratio(mouth):
    # compute the euclidean distances between the two sets of
    # vertical mouth landmarks (x, y)-coordinates
    A = dist.euclidean(mouth[2], mouth[10]) # 51, 59
    B = dist.euclidean(mouth[4], mouth[8]) # 53, 57

    # compute the euclidean distance between the horizontal
    # mouth landmark (x, y)-coordinates
    C = dist.euclidean(mouth[0], mouth[6]) # 49, 55

    # compute the mouth aspect ratio
    mar = (A + B) / (2.0 * C)
    marea = findArea(A,C)
    # return the mouth aspect ratio
    return mar,marea

In [11]:
def reye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical mouth landmarks (x, y)-coordinates
    A = dist.euclidean(eye[2], eye[6]) # 38, 42
    B = dist.euclidean(eye[3], eye[5]) # 39, 41
    # compute the euclidean distance between the horizontal
    # mouth landmark (x, y)-coordinates
    C = dist.euclidean(mouth[0], mouth[3]) # 36, 40

    # compute the mouth aspect ratio
    mar = (A + B) / (2.0 * C)
    marea = findArea(A,C)
    # return the mouth aspect ratio
    return mar,marea

In [12]:
def reye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical mouth landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5]) # 44, 48
    B = dist.euclidean(eye[3], eye[4]) # 45, 47
    # compute the euclidean distance between the horizontal
    # mouth landmark (x, y)-coordinates
    C = dist.euclidean(mouth[0], mouth[3]) # 43, 46

    # compute the mouth aspect ratio
    mar = (A + B) / (2.0 * C)
    marea = findArea(A,C)
    # return the mouth aspect ratio
    return mar,marea

In [1]:
from numpy.testing._private.utils import print_assert_equal
# import the necessary packages
from collections import OrderedDict
import numpy as np
import cv2
import argparse
import dlib
import imutils

facial_features_cordinates = {}

# define a dictionary that maps the indexes of the facial
# landmarks to specific face regions
FACIAL_LANDMARKS_INDEXES = OrderedDict([
    ("Mouth", (48, 68)),
    ("Right_Eye", (36, 42)),
    ("Left_Eye", (43, 48)),
])

# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-p", "--shape-predictor", required=True,
	help="/content/shape_predictor_68_face_landmarks.dat")
ap.add_argument("-i", "--image", required=True,
	help="path to input image")
args = vars(ap.parse_args())


def shape_to_numpy_array(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coordinates = np.zeros((68, 2), dtype=dtype)

    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coordinates[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coordinates


def visualize_facial_landmarks(image, shape, colors=None, alpha=0.75):
    # create two copies of the input image -- one for the
    # overlay and one for the final output image
    overlay = image.copy()
    output = image.copy()
    mar = 0
    rear = 0 
    lear = 0 
    # if the colors list is None, initialize it with a unique
    # color for each facial landmark region
    if colors is None:
        colors = [(19, 199, 109), (79, 76, 240), (230, 159, 23)
                  ]

    # loop over the facial landmark regions individually
    for (i, name) in enumerate(FACIAL_LANDMARKS_INDEXES.keys()):
        # grab the (x, y)-coordinates associated with the
        # face landmark
        (j, k) = FACIAL_LANDMARKS_INDEXES[name]
        pts = shape[j:k]
        facial_features_cordinates[name] = pts

        # check if are supposed to draw the jawline
        if name == "Mouth":
            # since the jawline is a non-enclosed facial region,
            # just draw lines between the (x, y)-coordinates
            mar,marea = mouth_aspect_ratio(pts)
            print("Mouth Area : "+str(marea))
        elif name == "Right_Eye":
          rear,marea = reye_aspect_ratio(pts)
          print("Right Eye Area : "+str(marea))

        # otherwise, compute the convex hull of the facial
        # landmark coordinates points and display it
        else :
          lear,marea = Lreye_aspect_ratio(pts)
          print("Left Eye Area : "+str(marea))
        hull = cv2.convexHull(pts)
        cv2.drawContours(overlay, [hull], -1, colors[i], -1)

    # apply the transparent overlay
    cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)

    # return the output image
    print(facial_features_cordinates)
    return output,mar,rear,lear

# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(args["shape_predictor"])

# load the input image, resize it, and convert it to grayscale
image = cv2.imread(args["image"])
image = imutils.resize(image, width=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# detect faces in the grayscale image
rects = detector(gray, 1)

# loop over the face detections
for (i, rect) in enumerate(rects):
    # determine the facial landmarks for the face region, then
    # convert the landmark (x, y)-coordinates to a NumPy array
    shape = predictor(gray, rect)
    shape = shape_to_numpy_array(shape)

    output,mar,lear,rear = visualize_facial_landmarks(image, shape)
    if (mar >0.75) and (lear < 0.3) and (rear < 0.3) :
      print("that perosn is sleeping")
    cv2.imshow("Image", output)
    cv2.waitKey(0)

usage: ipykernel_launcher.py [-h] [-p SHAPE_PREDICTOR] [-w WEBCAM]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-c319b863-d0b2-46f8-a83f-3d602aa199ab.json


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
